<a href="https://colab.research.google.com/github/ethantrang/algo-environmentday/blob/main/AlgoEnvironmentalDay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning GPT2-model for Article Generating


Import Google Drive folders

In [2]:
import os
os.chdir("/content/drive/MyDrive/EnvironmetalDay")

import neccessary module with corresponding version:
- transformer - get model and architect
- tokenizers - model to decode words

In [3]:
!pip3 install transformers==4.1.0
!pip3 install tokenizers==0.9.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 19.8 MB/s 
     |████████████████████████████████| 880 kB 61.5 MB/s 
     |████████████████████████████████| 2.9 MB 66.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=34d0338862d712326dc44255b82050fa1176cd428b6d8855baa56527856d74d1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Prepare training and evaluation dataset
Split the `total_data.csv` into two dataset train and eval with the ratio of 80/20 due to the size of the training data is quite small. 
1. the training dataset is saved into `train_data.csv`
2. the evaluation dataset is saved into `test_data.csv`

In [4]:
import transformers
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv("total_data.csv")

num = int(0.8 * len(data))

idxs = np.random.randint(0,len(data),len(data))
idxs_train = idxs[:num]
idxs_test = idxs[num:]

train,test = data.iloc[idxs_train], data.iloc[idxs_test]

train.to_csv("train_data.csv", index = False, sep = "\n")
test.to_csv("test_data.csv", index = False, sep = "\n")

print("Train dataset length: " + str(len(train)))
print("Test dataset length: " + str(len(test)))

Train dataset length: 17950
Test dataset length: 4488


Download GPT2 pretrained tokenizer model and set up training and eval file

In [5]:
from transformers import AutoTokenizer, GPT2Tokenizer

tokenizer_en = GPT2Tokenizer.from_pretrained("gpt2")

train_path = 'train_data.csv'
test_path = 'test_data.csv'

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

## Pretraining model

In [6]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
  train_dataset = TextDataset(tokenizer = tokenizer, file_path = train_path, block_size = 128)
  test_dataset = TextDataset(tokenizer = tokenizer, file_path = test_path, block_size = 128)

  data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)
  return train_dataset, test_dataset, data_collator
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer_en)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [9]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, GPT2LMHeadModel

# model = AutoModelWithLMHead.from_pretrained("vinai/phobert-base")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

training_args = TrainingArguments(
    output_dir = "gpt2", #đường dẫn thư mục lưu vào
    overwrite_output_dir = True,
    num_train_epochs = 3, #số epcoh huấn luyện
    per_device_train_batch_size = 8, #số batch huấn luyện
    per_device_eval_batch_size = 8, #số batch test
    eval_steps = 400, #số bước cập nhật giữa hai lần evaluate
    save_steps = 500, #số bước giữa hai lần checkpoint
    warmup_steps = 500, #số bước warmup cho scheduler
)

trainer = Trainer(
    model = model,
    tokenizer = tokenizer_en,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
  )

https://huggingface.co/gpt2-medium/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7t9rj4ni


Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

storing https://huggingface.co/gpt2-medium/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
creating metadata file for /root/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
loading configuration file https://huggingface.co/gpt2-medium/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

storing https://huggingface.co/gpt2-medium/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/6249eef5c8c1fcfccf9f36fc2e59301b109ac4036d8ebbee9c2b7f7e47f440bd.2538e2565f9e439a3668b981faf959c8b490b36dd631f3c4cd992519b2dd36f1
creating metadata file for /root/.cache/huggingface/transformers/6249eef5c8c1fcfccf9f36fc2e59301b109ac4036d8ebbee9c2b7f7e47f440bd.2538e2565f9e439a3668b981faf959c8b490b36dd631f3c4cd992519b2dd36f1
loading weights file https://huggingface.co/gpt2-medium/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/6249eef5c8c1fcfccf9f36fc2e59301b109ac4036d8ebbee9c2b7f7e47f440bd.2538e2565f9e439a3668b981faf959c8b490b36dd631f3c4cd992519b2dd36f1
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at gpt2-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadMod

In [7]:
transformers.logging.set_verbosity_info()

Train

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 114946
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 43107


Step,Training Loss
500,3.421478


Saving model checkpoint to gpt2/checkpoint-500
Configuration saved in gpt2/checkpoint-500/config.json
Model weights saved in gpt2/checkpoint-500/pytorch_model.bin


In [ ]:
trainer.save_model()

Using the pretrained model of huggingface through pipeline

In [11]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-medium')
set_seed(42)
for i in generator("WE CAN COME TOGETHER, ACT NOW TO RESTORE COASTAL ECOSYSTEMS,", max_length=100, num_return_sequences=5):
  print(i['generated_text'])

loading configuration file https://huggingface.co/gpt2-medium/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample"

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

storing https://huggingface.co/gpt2-medium/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/fee58641d7a73348d842afaa337d5a7763dad32beff8d9008bb3c3c847749d6b.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
creating metadata file for /root/.cache/huggingface/transformers/fee58641d7a73348d842afaa337d5a7763dad32beff8d9008bb3c3c847749d6b.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
https://huggingface.co/gpt2-medium/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp8zf4syqa


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

storing https://huggingface.co/gpt2-medium/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/23c853a0fcfc12c7d72ad4e922068b6982665b673f6de30b4c5cbe5bd70a2236.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/23c853a0fcfc12c7d72ad4e922068b6982665b673f6de30b4c5cbe5bd70a2236.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/gpt2-medium/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp6ogvn4ds


Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

storing https://huggingface.co/gpt2-medium/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/8e4f9a65085b1b4ae69ffac9a953a44249c9ea1e72e4a7816ee87b70081df038.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0
creating metadata file for /root/.cache/huggingface/transformers/8e4f9a65085b1b4ae69ffac9a953a44249c9ea1e72e4a7816ee87b70081df038.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0
loading file https://huggingface.co/gpt2-medium/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/fee58641d7a73348d842afaa337d5a7763dad32beff8d9008bb3c3c847749d6b.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
loading file https://huggingface.co/gpt2-medium/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/23c853a0fcfc12c7d72ad4e922068b6982665b673f6de30b4c5cbe5bd70a2236.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/gpt2-mediu

[{'generated_text': "WE CAN COME TOGETHER, ACT NOW TO RESTORE COASTAL ECOSYSTEMS, INDEPENDENCE, AND RESOURCE. NO TOGETHER WOULD BE TOO HARD TO BREAK THROUGH THE LOSS, BUT TOGETHER IT COULD TURN OUT BETTER OFF.\n\nFor the sake of our citizens' welfare and that of the entire planet, we can not allow ourselves to be divided like this, and it can only lead to more"},
 {'generated_text': 'WE CAN COME TOGETHER, ACT NOW TO RESTORE COASTAL ECOSYSTEMS, FORESEE THE PEACE THAT WE SURRENDER TO OUR CONDICTION OF WAR, AND END THE WAR ON CIVILIANS." — President Bush, June 6, 2006\n\n"I DON\'T KNOW WHAT THE HECK I\'M DOING. I DON\'T KNOW WHAT\'S GOING ON. I DON\'T KNOW WHAT WE\''},
 {'generated_text': 'WE CAN COME TOGETHER, ACT NOW TO RESTORE COASTAL ECOSYSTEMS, JOIN AMERICA." The statement goes on to urge all concerned to speak out against climate alarmism and to call for a halt to U.S. military bases and other activities that threaten public safety.\n\nThis is but the beginning of a campaign to stop

## Text Generating 

In [ ]:
prompt = "Tôi làm nhân viên lễ tân một khách sạn trên đường Hoàng Diệu."

In [ ]:
openai_generated_text_en_ids = tokenizer_en.encode(prompt, return_tensors = 'pt').to('cuda')
max_length = openai_generated_text_en_ids.shape[1]

print(max_length)

In [ ]:
input_ids = tokenizer_en.encode(prompt, return_tensors='pt').to('cuda')

### TopK Sampling

In [ ]:
%%time
model_vi = AutoModelWithLMHead.from_pretrained("./gpt2").to('cuda')
sample_outputs = model_vi.generate(input_ids, pad_token_id = 50256, do_sample = True, max_length=max_length, min_length = max_length, top_k = 40, num_return_sequences = 10)


In [ ]:
for i,sample_output in enumerate(sample_outputs):
  print(">>Generated text {}\n\n{}".format(i+1, str(tokenizer_en.decode(sample_output.tolist()))))
  print("\n---")

### Top-p nucleus sampling

In [ ]:
%%time
model_vi = AutoModelWithLMHead.from_pretrained("gpt2").to('cuda')
sample_outputs = model_vi.generate(input_ids, pad_token_id = 50256, do_sample = True, 
                                   max_length=max_length, 
                                   min_length = max_length, 
                                   repitition_penalty = 1.2,
                                   temperature = 0.7,
                                   top_k=50,
                                   top_p=0.95,
                                   num_return_sequences = 10)

In [ ]:
for i,sample_output in enumerate(sample_outputs):
  print(">>Generated text {}\n\n{}".format(i+1, str(tokenizer_en.decode(sample_output.tolist()))))
  print("\n---")